In [ ]:
# -*- coding: utf-8 -*-
import pandas as pd
from habanero import Crossref
from ratelimit import limits, sleep_and_retry
from tracking_grants import articles_f, cr_metadata_f, email, tool_name
from tracking_grants.utils.logging import logger
import json

In [ ]:
from tqdm.auto import tqdm

In [ ]:
# load articles
articles = pd.read_csv(articles_f, index_col="article_id")

In [4]:
cr = Crossref(mailto=email, ua_string=tool_name)

In [5]:
@sleep_and_retry
@limits(calls=10, period=1)
def query_crossref(cr, dois):
    return cr.works(dois)

In [6]:
dois = articles.DOI.tolist()
dois = dois[0:20]

dois_per_call = 10
r = list(range(0, len(dois), dois_per_call))
r = r + [len(dois)]

results = []
for ix in tqdm(range(len(r)-1), total=len(r)-1):
    response = query_crossref(cr, dois[r[ix]:r[ix+1]])
    results.extend(response)

In [7]:
with open(cr_metadata_f, 'w') as f:
    json.dump(results, f)

## Processing data

In [ ]:
direct_fields = ["ISSN", 'container-title', 'publisher', 'is-referenced-by-count', 'references-count', 'subject']
transform_fields = ["authors_count", ]
date_fields = ['created', 'deposited', 'indexed', 'published-online', 'issued']

In [13]:
for r in results:
    r['message']